In [1]:
from MyTT import *


In [2]:
import inspect
import importlib

# 假设你的 Python 文件名为 example_module.py
module_name = "MyTT"
module = importlib.import_module(module_name)

# 获取模块中的所有函数
functions = inspect.getmembers(module, inspect.isfunction)

for func_name, func in functions:
    print(f"函数名: {func_name}")
    
    # 获取函数的签名
    sig = inspect.signature(func)
    print(f"参数: {sig.parameters}")
    
    # 获取函数的返回注解
    return_annotation = sig.return_annotation
    if return_annotation is inspect.Signature.empty:
        return_annotation = "无返回值注解"
    print(f"返回值: {return_annotation}")
    print("-" * 40)


函数名: ABS
参数: OrderedDict({'S': <Parameter "S">})
返回值: 无返回值注解
----------------------------------------
函数名: ASI
参数: OrderedDict({'OPEN': <Parameter "OPEN">, 'CLOSE': <Parameter "CLOSE">, 'HIGH': <Parameter "HIGH">, 'LOW': <Parameter "LOW">, 'M1': <Parameter "M1=26">, 'M2': <Parameter "M2=10">})
返回值: 无返回值注解
----------------------------------------
函数名: ATR
参数: OrderedDict({'CLOSE': <Parameter "CLOSE">, 'HIGH': <Parameter "HIGH">, 'LOW': <Parameter "LOW">, 'N': <Parameter "N=20">})
返回值: 无返回值注解
----------------------------------------
函数名: AVEDEV
参数: OrderedDict({'S': <Parameter "S">, 'N': <Parameter "N">})
返回值: 无返回值注解
----------------------------------------
函数名: BARSLAST
参数: OrderedDict({'S': <Parameter "S">})
返回值: 无返回值注解
----------------------------------------
函数名: BARSLASTCOUNT
参数: OrderedDict({'S': <Parameter "S">})
返回值: 无返回值注解
----------------------------------------
函数名: BARSSINCEN
参数: OrderedDict({'S': <Parameter "S">, 'N': <Parameter "N">})
返回值: 无返回值注解
---------------------------

In [16]:
import ast

class FunctionAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.functions = []

    def visit_FunctionDef(self, node):
        func_info = {
            'name': node.name,
            'params': [arg.arg for arg in node.args.args],
            'returns_count': self.get_return_count(node)
        }
        self.functions.append(func_info)
        self.generic_visit(node)

    def get_return_count(self, node):
        return_count = 0
        for child in ast.walk(node):
            if isinstance(child, ast.Return):
                if isinstance(child.value, ast.Tuple):
                    return_count = max(return_count, len(child.value.elts))
                else:
                    return_count = max(return_count, 1)
        return return_count

def analyze_functions(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        tree = ast.parse(file.read())
    analyzer = FunctionAnalyzer()
    analyzer.visit(tree)
    return analyzer.functions

# 示例使用
file_path = 'MyTT.py'  # 替换为你的 Python 文件路径
functions_info = analyze_functions(file_path)
func_info={}
for func in functions_info:
    func_info[func['name']]={'params':func['params'],'returns_count':func['returns_count']}
    # print(func['name'],func_info[func['name']])
    # print(f"函数名: {func['name']}")
    # print(f"参数: {func['args']}")
    # print(f"返回值个数: {func['returns']}")
    # print("-" * 40)
print(func_info)

{'RD': {'params': ['N', 'D'], 'returns_count': 1}, 'RET': {'params': ['S', 'N'], 'returns_count': 1}, 'ABS': {'params': ['S'], 'returns_count': 1}, 'LN': {'params': ['S'], 'returns_count': 1}, 'POW': {'params': ['S', 'N'], 'returns_count': 1}, 'SQRT': {'params': ['S'], 'returns_count': 1}, 'SIN': {'params': ['S'], 'returns_count': 1}, 'COS': {'params': ['S'], 'returns_count': 1}, 'TAN': {'params': ['S'], 'returns_count': 1}, 'MAX': {'params': ['S1', 'S2'], 'returns_count': 1}, 'MIN': {'params': ['S1', 'S2'], 'returns_count': 1}, 'IF': {'params': ['S', 'A', 'B'], 'returns_count': 1}, 'REF': {'params': ['S', 'N'], 'returns_count': 1}, 'DIFF': {'params': ['S', 'N'], 'returns_count': 1}, 'STD': {'params': ['S', 'N'], 'returns_count': 1}, 'SUM': {'params': ['S', 'N'], 'returns_count': 1}, 'CONST': {'params': ['S'], 'returns_count': 1}, 'HHV': {'params': ['S', 'N'], 'returns_count': 1}, 'LLV': {'params': ['S', 'N'], 'returns_count': 1}, 'HHVBARS': {'params': ['S', 'N'], 'returns_count': 1}, 

In [8]:
# 获取函数的签名
sig = inspect.signature(MA)
# print(f"参数: {sig.parameters}")
parameters = sig.parameters
list(parameters.keys())

['S', 'N']

In [28]:
import talib
from talib import abstract
info = abstract.Function("SMA").info
info

{'name': 'SMA',
 'group': 'Overlap Studies',
 'display_name': 'Simple Moving Average',
 'function_flags': ['Output scale same as input'],
 'input_names': OrderedDict([('price', 'close')]),
 'parameters': OrderedDict([('timeperiod', 30)]),
 'output_flags': OrderedDict([('real', ['Line'])]),
 'output_names': ['real']}

In [64]:
from vectorbt.indicators import IndicatorBase
import vectorbt as vbt
from vectorbt import _typing as tp
from vectorbt.utils.config import merge_dicts
from vectorbt.base.reshape_fns import to_2d_array
import MyTT


class MYTT_ADD(vbt.IndicatorFactory):
    PARAMS = ('S','S1','S2','OPEN','CLOSE','HIGH','LOW','VOLUME')
    
    FUNC_INFO={'RD': {'params': ['N', 'D'], 'returns_count': 1}, 'RET': {'params': ['S', 'N'], 'returns_count': 1}, 'ABS': {'params': ['S'], 'returns_count': 1}, 'LN': {'params': ['S'], 'returns_count': 1}, 'POW': {'params': ['S', 'N'], 'returns_count': 1}, 'SQRT': {'params': ['S'], 'returns_count': 1}, 'SIN': {'params': ['S'], 'returns_count': 1}, 'COS': {'params': ['S'], 'returns_count': 1}, 'TAN': {'params': ['S'], 'returns_count': 1}, 'MAX': {'params': ['S1', 'S2'], 'returns_count': 1}, 'MIN': {'params': ['S1', 'S2'], 'returns_count': 1}, 'IF': {'params': ['S', 'A', 'B'], 'returns_count': 1}, 'REF': {'params': ['S', 'N'], 'returns_count': 1}, 'DIFF': {'params': ['S', 'N'], 'returns_count': 1}, 'STD': {'params': ['S', 'N'], 'returns_count': 1}, 'SUM': {'params': ['S', 'N'], 'returns_count': 1}, 'CONST': {'params': ['S'], 'returns_count': 1}, 'HHV': {'params': ['S', 'N'], 'returns_count': 1}, 'LLV': {'params': ['S', 'N'], 'returns_count': 1}, 'HHVBARS': {'params': ['S', 'N'], 'returns_count': 1}, 'LLVBARS': {'params': ['S', 'N'], 'returns_count': 1}, 'MA': {'params': ['S', 'N'], 'returns_count': 1}, 'EMA': {'params': ['S', 'N'], 'returns_count': 1}, 'SMA': {'params': ['S', 'N', 'M'], 'returns_count': 1}, 'WMA': {'params': ['S', 'N'], 'returns_count': 1}, 'DMA': {'params': ['S', 'A'], 'returns_count': 1}, 'AVEDEV': {'params': ['S', 'N'], 'returns_count': 1}, 'SLOPE': {'params': ['S', 'N'], 'returns_count': 1}, 'FORCAST': {'params': ['S', 'N'], 'returns_count': 1}, 'LAST': {'params': ['S', 'A', 'B'], 'returns_count': 1}, 'COUNT': {'params': ['S', 'N'], 'returns_count': 1}, 'EVERY': {'params': ['S', 'N'], 'returns_count': 1}, 'EXIST': {'params': ['S', 'N'], 'returns_count': 1}, 'FILTER': {'params': ['S', 'N'], 'returns_count': 1}, 'BARSLAST': {'params': ['S'], 'returns_count': 1}, 'BARSLASTCOUNT': {'params': ['S'], 'returns_count': 1}, 'BARSSINCEN': {'params': ['S', 'N'], 'returns_count': 1}, 'CROSS': {'params': ['S1', 'S2'], 'returns_count': 1}, 'LONGCROSS': {'params': ['S1', 'S2', 'N'], 'returns_count': 1}, 'VALUEWHEN': {'params': ['S', 'X'], 'returns_count': 1}, 'BETWEEN': {'params': ['S', 'A', 'B'], 'returns_count': 1}, 'TOPRANGE': {'params': ['S'], 'returns_count': 1}, 'LOWRANGE': {'params': ['S'], 'returns_count': 1}, 'MACD': {'params': ['CLOSE', 'SHORT', 'LONG', 'M'], 'returns_count': 3}, 'KDJ': {'params': ['CLOSE', 'HIGH', 'LOW', 'N', 'M1', 'M2'], 'returns_count': 3}, 'RSI': {'params': ['CLOSE', 'N'], 'returns_count': 1}, 'WR': {'params': ['CLOSE', 'HIGH', 'LOW', 'N', 'N1'], 'returns_count': 2}, 'BIAS': {'params': ['CLOSE', 'L1', 'L2', 'L3'], 'returns_count': 3}, 'BOLL': {'params': ['CLOSE', 'N', 'P'], 'returns_count': 3}, 'PSY': {'params': ['CLOSE', 'N', 'M'], 'returns_count': 2}, 'CCI': {'params': ['CLOSE', 'HIGH', 'LOW', 'N'], 'returns_count': 1}, 'ATR': {'params': ['CLOSE', 'HIGH', 'LOW', 'N'], 'returns_count': 1}, 'BBI': {'params': ['CLOSE', 'M1', 'M2', 'M3', 'M4'], 'returns_count': 1}, 'DMI': {'params': ['CLOSE', 'HIGH', 'LOW', 'M1', 'M2'], 'returns_count': 4}, 'TAQ': {'params': ['HIGH', 'LOW', 'N'], 'returns_count': 3}, 'KTN': {'params': ['CLOSE', 'HIGH', 'LOW', 'N', 'M'], 'returns_count': 3}, 'TRIX': {'params': ['CLOSE', 'M1', 'M2'], 'returns_count': 2}, 'VR': {'params': ['CLOSE', 'VOL', 'M1'], 'returns_count': 1}, 'CR': {'params': ['CLOSE', 'HIGH', 'LOW', 'N'], 'returns_count': 1}, 'EMV': {'params': ['HIGH', 'LOW', 'VOL', 'N', 'M'], 'returns_count': 2}, 'DPO': {'params': ['CLOSE', 'M1', 'M2', 'M3'], 'returns_count': 2}, 'BRAR': {'params': ['OPEN', 'CLOSE', 'HIGH', 'LOW', 'M1'], 'returns_count': 2}, 'DFMA': {'params': ['CLOSE', 'N1', 'N2', 'M'], 'returns_count': 2}, 'MTM': {'params': ['CLOSE', 'N', 'M'], 'returns_count': 2}, 'MASS': {'params': ['HIGH', 'LOW', 'N1', 'N2', 'M'], 'returns_count': 2}, 'ROC': {'params': ['CLOSE', 'N', 'M'], 'returns_count': 2}, 'EXPMA': {'params': ['CLOSE', 'N1', 'N2'], 'returns_count': 2}, 'OBV': {'params': ['CLOSE', 'VOL'], 'returns_count': 1}, 'MFI': {'params': ['CLOSE', 'HIGH', 'LOW', 'VOL', 'N'], 'returns_count': 1}, 'ASI': {'params': ['OPEN', 'CLOSE', 'HIGH', 'LOW', 'M1', 'M2'], 'returns_count': 2}, 'XSII': {'params': ['CLOSE', 'HIGH', 'LOW', 'N', 'M'], 'returns_count': 4}}
    
    def __init__(self,
                 class_name: str = 'CustomIndicator',
                 class_docstring: str = '',
                 module_name: tp.Optional[str] = __name__,
                 short_name: tp.Optional[str] = None,
                 prepend_name: bool = True,
                 input_names: tp.Optional[tp.Sequence[str]] = None,
                 param_names: tp.Optional[tp.Sequence[str]] = None,
                 in_output_names: tp.Optional[tp.Sequence[str]] = None,
                 output_names: tp.Optional[tp.Sequence[str]] = None,
                 output_flags: tp.KwargsLike = None,
                 custom_output_props: tp.KwargsLike = None,
                 attr_settings: tp.KwargsLike = None,
                 metrics: tp.Optional[tp.Kwargs] = None,
                 stats_defaults: tp.Union[None, tp.Callable, tp.Kwargs] = None,
                 subplots: tp.Optional[tp.Kwargs] = None,
                 plots_defaults: tp.Union[None, tp.Callable, tp.Kwargs] = None) -> None:
        super().__init__(class_name=class_name,
                         class_docstring=class_docstring,
                         module_name=module_name,
                         short_name=short_name,
                         prepend_name=prepend_name,
                         input_names=input_names,
                         param_names=param_names,
                         in_output_names=in_output_names,
                         output_names=output_names,
                         output_flags=output_flags,
                         custom_output_props=custom_output_props,
                         attr_settings=attr_settings,
                         metrics=metrics,
                         stats_defaults=stats_defaults,
                         subplots=subplots,
                         plots_defaults=plots_defaults)
    
    @classmethod
    def from_mytt(cls, func_name: str, init_kwargs: tp.KwargsLike = None, **kwargs) -> tp.Type[IndicatorBase]:
        mytt_func = getattr(MyTT, func_name)
        input_names = [ p for p in cls.FUNC_INFO[func_name]['params'] if p not in cls.PARAMS]
        class_name = "MyTT"
        class_docstring = "{}, {}".format("MyTT", func_name)
        param_names = [ p for p in cls.FUNC_INFO[func_name]['params'] if p in cls.PARAMS]
        output_names = [f"real_{i}" for i in range(cls.FUNC_INFO[func_name]['returns_count'])]
        output_flags = {}
        

        
        def apply_func(input_list: tp.List[tp.AnyArray],
                       in_output_tuple: tp.Tuple[tp.AnyArray, ...],
                       param_tuple: tp.Tuple[tp.Param, ...],
                       **kwargs) -> tp.Union[tp.Array2d, tp.List[tp.Array2d]]:
            # TA-Lib functions can only process 1-dim arrays
            n_input_cols = input_list[0].shape[1]
            outputs = []
            for col in range(n_input_cols):
                output = mytt_func(
                    *map(lambda x: x[:, col], input_list),
                    *param_tuple,
                    **kwargs
                )
                outputs.append(output)
            if isinstance(outputs[0], tuple):  # multiple outputs
                outputs = list(zip(*outputs))
                return list(map(np.column_stack, outputs))
            return np.column_stack(outputs)

        MyTTIndicator = cls(
            **merge_dicts(
                dict(
                    class_name=class_name,
                    class_docstring=class_docstring,
                    input_names=input_names,
                    param_names=param_names,
                    output_names=output_names,
                    output_flags=output_flags
                ),
                init_kwargs
            )
        ).from_apply_func(
            apply_func,
            pass_packed=True,
            **{p :5 for p in param_names},
            **kwargs
        )
        return MyTTIndicator
        



In [70]:
import numpy as np
MA = MYTT_ADD.from_mytt("EMA")
# 生成一个包含100个随机数的数组
random_array = np.random.rand(600)
MA.run(random_array, np.arange(5,20)).real_0

mytt_S,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602,0.547602
1,0.399308,0.420493,0.436381,0.448739,0.458626,0.466714,0.473455,0.479159,0.484047,0.488284,0.491992,0.495263,0.498171,0.500772,0.503114
2,0.413274,0.426411,0.437588,0.447065,0.455142,0.462076,0.468080,0.473320,0.477927,0.482007,0.485644,0.488903,0.491841,0.494502,0.496923
3,0.421782,0.429950,0.437891,0.445228,0.451873,0.457844,0.463200,0.468009,0.472338,0.476246,0.479788,0.483009,0.485948,0.488639,0.491111
4,0.399508,0.408524,0.417158,0.425169,0.432490,0.439138,0.445160,0.450617,0.455569,0.460075,0.464185,0.467944,0.471394,0.474567,0.477496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.482694,0.514776,0.536546,0.551305,0.561257,0.567872,0.572144,0.574746,0.576144,0.576667,0.576548,0.575954,0.575011,0.573811,0.572424
596,0.445375,0.473623,0.495094,0.511179,0.523153,0.532030,0.538576,0.543360,0.546801,0.549210,0.550821,0.551811,0.552314,0.552435,0.552255
597,0.360292,0.392624,0.418852,0.439834,0.456547,0.469865,0.480501,0.489016,0.495847,0.501332,0.505734,0.509260,0.512071,0.514297,0.516043
598,0.559319,0.553981,0.553482,0.554843,0.556713,0.558503,0.559980,0.561071,0.561780,0.562138,0.562189,0.561979,0.561549,0.560937,0.560176


In [21]:
import numpy as np
from MyTT import SMA

# 生成一个包含100个随机数的数组
random_array = np.random.rand(100)
SMA(random_array, 5)

array([0.76048416, 0.70058071, 0.58066767, 0.54268448, 0.48803094,
       0.5759651 , 0.53269921, 0.5168355 , 0.51367662, 0.5325532 ,
       0.58978851, 0.52642943, 0.45352941, 0.5094756 , 0.52708527,
       0.59075942, 0.50405296, 0.56297602, 0.60663027, 0.65109407,
       0.5234126 , 0.52953378, 0.44283099, 0.44456802, 0.37507129,
       0.41553673, 0.46334045, 0.42337899, 0.39584356, 0.49487225,
       0.46472179, 0.52328831, 0.48264947, 0.56842153, 0.53593222,
       0.43455384, 0.37942749, 0.43557764, 0.53484291, 0.5427464 ,
       0.50410519, 0.44862294, 0.50009983, 0.4737394 , 0.40857101,
       0.52478577, 0.57288047, 0.5243092 , 0.543634  , 0.43818577,
       0.46793505, 0.42099608, 0.35644862, 0.45902444, 0.43017701,
       0.37802014, 0.33050973, 0.4181956 , 0.47821507, 0.50824194,
       0.44176291, 0.54018728, 0.50625066, 0.56344645, 0.62574442,
       0.59570444, 0.61251161, 0.59764143, 0.58491857, 0.55669515,
       0.57848271, 0.5562516 , 0.49847157, 0.40920916, 0.48076

In [25]:
import talib
import vectorbt as vbt
SMA = vbt.IndicatorFactory.from_talib('SMA')
SMA.run(random_array, 5).real

0          NaN
1          NaN
2          NaN
3          NaN
4     0.396527
        ...   
95    0.410654
96    0.268651
97    0.395467
98    0.463723
99    0.522710
Name: 5, Length: 100, dtype: float64